<a href="https://colab.research.google.com/github/imhariharanb/Horticbot_Pest/blob/main/Pest_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# to ignore the warnings
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('https://drive.google.com/file/d/13QIckn11NCsYqu9TOWAKLU32A-LXKNAs/view?usp=sharing')

In [ ]:
!unzip 'https://drive.google.com/drive/folders/14YENWFqNVGVhsIgwB-Wa-pk4XxywmK1H?usp=drive_link'

unzip:  cannot find or open https://drive.google.com/drive/folders/14YENWFqNVGVhsIgwB-Wa-pk4XxywmK1H?usp=drive_link, https://drive.google.com/drive/folders/14YENWFqNVGVhsIgwB-Wa-pk4XxywmK1H?usp=drive_link.zip or https://drive.google.com/drive/folders/14YENWFqNVGVhsIgwB-Wa-pk4XxywmK1H?usp=drive_link.ZIP.

No zipfiles found.


In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
import glob
import matplotlib.pyplot as plt

In [ ]:
train_path  = "/content/pest/train"
#valid_path  = ""
test_path   = "/content/pest/test"
print("splited")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import vgg16
from keras.models import Model
from keras.layers import Dense, MaxPool2D, Conv2D
import keras

In [ ]:
train_data_gen = ImageDataGenerator(preprocessing_function= vgg16.preprocess_input , zoom_range= 0.2, horizontal_flip= True, shear_range= 0.2 , rescale= 1./255)
train = train_data_gen.flow_from_directory(directory= train_path , target_size=(224,224))

In [ ]:
test_data_gen = ImageDataGenerator(preprocessing_function= vgg16.preprocess_input, rescale= 1./255 )
test = train_data_gen.flow_from_directory(directory= test_path , target_size=(224,224), shuffle= False)

In [ ]:
train.class_indices

In [ ]:
class_type = {0:'aphids',
 1:'armyworm',
 2:'beetle',
 3:'bollworm',
 4:'grasshopper',
 5:'mites',
 6:'mosquito',
 7:'sawfly',
 8:'stem_borer'}

In [ ]:
t_img , label = train.next()

In [ ]:
def plotImages(img_arr, label):

  for im, l in zip(img_arr,label) :
    plt.figure(figsize= (5,5))
    plt.imshow(im, cmap = 'gray')
    plt.title(im.shape)
    plt.axis = False
    plt.show()

In [ ]:
plotImages(t_img, label)

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.layers import Flatten , Dense, Dropout , MaxPool2D

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.layers import Flatten , Dense, Dropout , MaxPool2D

In [ ]:
vgg = VGG16( input_shape=(224,224,3), include_top= False) # include_top will consider the new weights

In [ ]:
for layer in vgg.layers:           # Dont Train the parameters again
  layer.trainable = False

In [ ]:
x = Flatten()(vgg.output)
x = Dense(units=9, activation='softmax', name = 'predictions')(x)

model = Model(vgg.input, x)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# implementing early stopping and model check point

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

es = EarlyStopping(monitor= "val_accuracy" , min_delta= 0.01, patience= 3, verbose=1)
mc = ModelCheckpoint(filepath="bestmodel.h5", monitor="val_accuracy", verbose=1, save_best_only= False)

In [ ]:
#hist = model.fit_generator(train, steps_per_epoch= 10, epochs= 8, validation_data= valid , validation_steps= 32)
hist = model.fit_generator(train, steps_per_epoch= 10, epochs=20 , validation_data= test , validation_steps= 32, callbacks=[mc])

In [ ]:
## load only the best model
from keras.models import load_model
model = load_model("bestmodel.h5")

In [ ]:
h = hist.history
h.keys()

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'] , c = "red")
plt.title("acc vs v-acc")
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'] , c = "red")
plt.title("loss vs v-loss")
plt.show()

In [ ]:
# checking out the accuracy of our model

acc = model.evaluate_generator(generator= test)[1]

print(f"The accuracy of your model is = {acc} %")

In [ ]:
from keras.preprocessing import image

def get_img_array(img_path):
  """
  Input : Takes in image path as input
  Output : Gives out Pre-Processed image
  """
  path = img_path

  img = keras.utils.load_img(path, target_size=(224,224,3))

  img = keras.utils.img_to_array(img)/255
  img = np.expand_dims(img , axis= 0 )

  return img

In [ ]:
# path for that new image. ( you can take it either from google or any other scource)

path = "/content/pest/train/bollworm/jpg_11.jpg"       # you can add any image path

#predictions: path:- provide any image from google or provide image from all image folder
img = get_img_array(path)

res = class_type[np.argmax(model.predict(img))]
print(f"The given  image is of type = {res}")
print()

# to display the image
plt.imshow(img[0], cmap = "gray")
plt.title("input image")
plt.show()

In [ ]:

path = "/content/pest/train/beetle/jpg_1.jpg"       # you can add any image path

#predictions: path:- provide any image from google or provide image from all image folder
img = get_img_array(path)

res = class_type[np.argmax(model.predict(img))]
print(f"The given  image is of type = {res}")
print()

# to display the image
plt.imshow(img[0], cmap = "gray")
plt.title("input image")
plt.show()

In [ ]:

path = "/content/pest/train/aphids/jpg_1.jpg"       # you can add any image path

#predictions: path:- provide any image from google or provide image from all image folder
img = get_img_array(path)

res = class_type[np.argmax(model.predict(img))]
print(f"The given  image is of type = {res}")
print()

# to display the image
plt.imshow(img[0], cmap = "gray")
plt.title("input image")
plt.show()

In [ ]:

path = "/content/pest/train/armyworm/jpg_11.jpg"       # you can add any image path

#predictions: path:- provide any image from google or provide image from all image folder
img = get_img_array(path)

res = class_type[np.argmax(model.predict(img))]
print(f"The given  image is of type = {res}")
print()

# to display the image
plt.imshow(img[0], cmap = "gray")
plt.title("input image")
plt.show()

In [ ]:

path = "/content/pest/train/grasshopper/jpg_1.jpg"       # you can add any image path

#predictions: path:- provide any image from google or provide image from all image folder
img = get_img_array(path)

res = class_type[np.argmax(model.predict(img))]
print(f"The given  image is of type = {res}")
print()

# to display the image
plt.imshow(img[0], cmap = "gray")
plt.title("input image")
plt.show()

In [ ]:

path = "/content/pest/train/mites/jpg_1.jpg"       # you can add any image path

#predictions: path:- provide any image from google or provide image from all image folder
img = get_img_array(path)

res = class_type[np.argmax(model.predict(img))]
print(f"The given  image is of type = {res}")
print()

# to display the image
plt.imshow(img[0], cmap = "gray")
plt.title("input image")
plt.show()

In [ ]:

path = "/content/pest/train/mosquito/jpg_1.jpg"       # you can add any image path

#predictions: path:- provide any image from google or provide image from all image folder
img = get_img_array(path)

res = class_type[np.argmax(model.predict(img))]
print(f"The given  image is of type = {res}")
print()

# to display the image
plt.imshow(img[0], cmap = "gray")
plt.title("input image")
plt.show()

In [ ]:

path = "/content/pest/train/sawfly/jpg_1.jpg"       # you can add any image path

#predictions: path:- provide any image from google or provide image from all image folder
img = get_img_array(path)

res = class_type[np.argmax(model.predict(img))]
print(f"The given  image is of type = {res}")
print()

# to display the image
plt.imshow(img[0], cmap = "gray")
plt.title("input image")
plt.show()

In [ ]:

path = "/content/pest/train/stem_borer/jpg_1.jpg"       # you can add any image path

#predictions: path:- provide any image from google or provide image from all image folder
img = get_img_array(path)

res = class_type[np.argmax(model.predict(img))]
print(f"The given  image is of type = {res}")
print()

# to display the image
plt.imshow(img[0], cmap = "gray")
plt.title("input image")
plt.show()